In [4]:
import urllib

import optuna
from packaging import version

import tensorflow as tf
from tensorflow.keras.datasets import mnist


# TODO(crcrpar): Remove the below three lines once everything is ok.
# Register a global custom opener to avoid HTTP Error 403: Forbidden when downloading MNIST.
opener = urllib.request.build_opener()
opener.addheaders = [("User-agent", "Mozilla/5.0")]
urllib.request.install_opener(opener)


if version.parse(tf.__version__) < version.parse("2.0.0"):
    raise RuntimeError("tensorflow>=2.0.0 is required for this example.")

N_TRAIN_EXAMPLES = 3000
N_VALID_EXAMPLES = 1000
BATCHSIZE = 128
CLASSES = 10
EPOCHS = 1


def create_model(trial):
    # We optimize the numbers of layers, their units and weight decay parameter.
    n_layers = trial.suggest_int("n_layers", 1, 3)
    weight_decay = trial.suggest_float("weight_decay", 1e-10, 1e-3, log=True)
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten())
    for i in range(n_layers):
        num_hidden = trial.suggest_int("n_units_l{}".format(i), n_layers, n_layers, log=True)
        model.add(
            tf.keras.layers.Dense(
                num_hidden,
                activation="relu",
                kernel_regularizer=tf.keras.regularizers.l2(weight_decay),
            )
        )
    model.add(
        tf.keras.layers.Dense(CLASSES, kernel_regularizer=tf.keras.regularizers.l2(weight_decay))
    )
    return model


def create_optimizer(trial):
    # We optimize the choice of optimizers as well as their parameters.
    kwargs = {}
    optimizer_options = ["RMSprop", "Adam", "SGD"]
    optimizer_selected = trial.suggest_categorical("optimizer", optimizer_options)
    if optimizer_selected == "RMSprop":
        kwargs["learning_rate"] = trial.suggest_float(
            "rmsprop_learning_rate", 1e-5, 1e-1, log=True
        )
        kwargs["decay"] = trial.suggest_float("rmsprop_decay", 0.85, 0.99)
        kwargs["momentum"] = trial.suggest_float("rmsprop_momentum", 1e-5, 1e-1, log=True)
    elif optimizer_selected == "Adam":
        kwargs["learning_rate"] = trial.suggest_float("adam_learning_rate", 1e-5, 1e-1, log=True)
    elif optimizer_selected == "SGD":
        kwargs["learning_rate"] = trial.suggest_float(
            "sgd_opt_learning_rate", 1e-5, 1e-1, log=True
        )
        kwargs["momentum"] = trial.suggest_float("sgd_opt_momentum", 1e-5, 1e-1, log=True)

    optimizer = getattr(tf.optimizers, optimizer_selected)(**kwargs)
    return optimizer


def learn(model, optimizer, dataset, mode="eval"):
    accuracy = tf.metrics.Accuracy("accuracy", dtype=tf.float32)

    for batch, (images, labels) in enumerate(dataset):
        with tf.GradientTape() as tape:
            logits = model(images, training=(mode == "train"))
            loss_value = tf.reduce_mean(
                tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels)
            )
            if mode == "eval":
                accuracy(
                    tf.argmax(logits, axis=1, output_type=tf.int64), tf.cast(labels, tf.int64)
                )
            else:
                grads = tape.gradient(loss_value, model.variables)
                optimizer.apply_gradients(zip(grads, model.variables))

    if mode == "eval":
        return accuracy


def get_mnist():
    (x_train, y_train), (x_valid, y_valid) = mnist.load_data()
    x_train = x_train.astype("float32") / 255
    x_valid = x_valid.astype("float32") / 255

    y_train = y_train.astype("int32")
    y_valid = y_valid.astype("int32")

    train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    train_ds = train_ds.shuffle(60000).batch(BATCHSIZE).take(N_TRAIN_EXAMPLES)

    valid_ds = tf.data.Dataset.from_tensor_slices((x_valid, y_valid))
    valid_ds = valid_ds.shuffle(10000).batch(BATCHSIZE).take(N_VALID_EXAMPLES)
    return train_ds, valid_ds


# FYI: Objective functions can take additional arguments
# (https://optuna.readthedocs.io/en/stable/faq.html#objective-func-additional-args).
def objective(trial):
    # Get MNIST data.
    train_ds, valid_ds = get_mnist()

    # Build model and optimizer.
    model = create_model(trial)
    optimizer = create_optimizer(trial)

    # Training and validating cycle.
    with tf.device("/cpu:0"):
        for _ in range(EPOCHS):
            learn(model, optimizer, train_ds, "train")

        accuracy = learn(model, optimizer, valid_ds, "eval")

    # Return last validation accuracy.
    return accuracy.result()


if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100)

    print("Number of finished trials: ", len(study.trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2021-07-28 05:41:08,546] A new study created in memory with name: no-name-924226be-7ff5-4406-bd42-00c17c9ec020
[I 2021-07-28 05:41:11,279] Trial 0 finished with value: 0.19449999928474426 and parameters: {'n_layers': 1, 'weight_decay': 3.4648512604364385e-08, 'n_units_l0': 1, 'optimizer': 'RMSprop', 'rmsprop_learning_rate': 0.08799759878598154, 'rmsprop_decay': 0.9055406256660743, 'rmsprop_momentum': 0.00017331316559001967}. Best is trial 0 with value: 0.19449999928474426.
[I 2021-07-28 05:41:14,216] Trial 1 finished with value: 0.6080999970436096 and parameters: {'n_layers': 3, 'weight_decay': 7.760836538779416e-10, 'n_units_l0': 3, 'n_units_l1': 3, 'n_units_l2': 3, 'optimizer': 'Adam', 'adam_learning_rate': 0.005101949093151569}. Best is trial 1 with value: 0.6080999970436096.
[I 2021-07-28 05:41:17,342] Trial 2 finished with value: 0.10140000283718109 and parameters: {'n_layers': 3, 'weight_decay': 8.93588147595707e-05, 'n_units_l0': 3, 'n_units_l1': 3, 'n_units_l2': 3, 'optimize

KeyboardInterrupt: 

In [4]:
import optuna


def objective(trial):
    x = trial.suggest_float("x", 0, 10)
    return x ** 2


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=15)

[I 2021-07-29 03:06:49,035] A new study created in memory with name: no-name-462d6a40-083d-4835-b3b5-e7afac8b9aed
[I 2021-07-29 03:06:49,037] Trial 0 finished with value: 6.841872029275124 and parameters: {'x': 2.6156972357815276}. Best is trial 0 with value: 6.841872029275124.
[I 2021-07-29 03:06:49,038] Trial 1 finished with value: 33.79825764932452 and parameters: {'x': 5.8136268928547965}. Best is trial 1 with value: 33.79825764932452.
[I 2021-07-29 03:06:49,039] Trial 2 finished with value: 51.402552329557444 and parameters: {'x': 7.169557331492471}. Best is trial 2 with value: 51.402552329557444.
[I 2021-07-29 03:06:49,040] Trial 3 finished with value: 79.42502116043299 and parameters: {'x': 8.91207165368597}. Best is trial 3 with value: 79.42502116043299.
[I 2021-07-29 03:06:49,042] Trial 4 finished with value: 13.10152231402616 and parameters: {'x': 3.6196025077384064}. Best is trial 3 with value: 79.42502116043299.
[I 2021-07-29 03:06:49,043] Trial 5 finished with value: 12.96